In [18]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read historical wheat data 
- for Argentina on municipality level
- from 1969-2022
- SOURCE: https://datosestimaciones.magyp.gob.ar/

In [4]:
df = (pd
      .read_csv("data/crop/historical_wheat_data_argentina.csv", encoding="latin-1", sep=";")
      .drop(columns={"Cultivo", "idProvincia", "idDepartamento", "Sup. Sembrada"}))

df.columns = ["year", "province", "municipality", "harvested area [ha]", "production [t]", "yield [kg/ha]"]

"""
"Campaña" is based on USDA marketing year defintion. Argentinian wheat season spans from June-Dec. 
The following paragraph from USDA from April 2023 indicates how the first year number (e.g. XXXX in XXXX/YY) is relevant for our crop forecasts.

Post estimates Argentine wheat production to rebound to 19.5 million metric tons (MMT) in marketing year (MY) 2023/24, 
resulting in wheat exports of 13.7 MMT (including wheat flour as its wheat equivalent). However after the severe drought in MY 2022/23, 
more rains are needed to recharge soil moisture profiles before the June planting window (from: https://www.fas.usda.gov/data/argentina-grain-and-feed-annual-9)
"""
df["year"] = df["year"].str[:4].astype(int)

#df = df.loc[df["year"] >= 1973].reset_index(drop=True) # because of SCM availability

df.head()

,year,province,municipality,harvested area [ha],production [t],yield [kg/ha]
0,1969,BUENOS AIRES,25 DE MAYO,42000,71400,1700
1,1970,BUENOS AIRES,25 DE MAYO,36000,54000,1500
2,1971,BUENOS AIRES,25 DE MAYO,30000,39000,1300
3,1972,BUENOS AIRES,25 DE MAYO,41000,73600,1795
4,1973,BUENOS AIRES,25 DE MAYO,6000,9000,1500


We filter by provinces with relevant production

In [5]:
production_by_province = df.groupby("province")["production [t]"].sum().reset_index().sort_values(by="production [t]", ascending=False).reset_index(drop=True)
production_by_province["production [t]"] = np.round(100 * production_by_province["production [t]"] / df["production [t]"].sum(), 2)
production_by_province.head(10)

,province,production [t]
0,BUENOS AIRES,57.50
1,SANTA FE,15.87
2,CORDOBA,13.20
3,LA PAMPA,5.17
4,ENTRE RIOS,4.61
5,SANTIAGO DEL ESTERO,1.57
6,TUCUMAN,0.64
7,CHACO,0.61
8,SALTA,0.52
9,CATAMARCA,0.13


Buenos Aires, Santa Fe, Cordoba, Entre Rios, and La Pampa contribute most of Argentinian wheat production. We will only focus on these 5 provinces from now on. In addition, these are the provinces with similar soil and climate conditions (Espinal, Pampas)

In [6]:
df = df.loc[df["province"].isin(['BUENOS AIRES', 'CORDOBA', 'ENTRE RIOS', 'LA PAMPA', 'SANTA FE'])].reset_index(drop=True)

Filter by municipalities with complete time series

In [13]:
df_province = df[df.groupby(["province", "municipality"])[['province', "municipality"]].transform('size') == len(df["year"].unique().tolist())].reset_index(drop=True)

Now we need to aggregate to national level by year. Yield is weighted with harvested area.

In [14]:
df_national = df.copy()
df_national["yield weighted"] = df_national["yield [kg/ha]"] * df["harvested area [ha]"]
df_national = df_national.groupby("year").agg({"harvested area [ha]":"sum", "yield weighted":"sum"})
df_national["yield [kg/ha]"] = df_national["yield weighted"] / df_national["harvested area [ha]"]

df_national = df_national[["harvested area [ha]", "yield [kg/ha]"]].reset_index()
df_national.head()

,year,harvested area [ha],yield [kg/ha]
0,1969,5106050,1359.034983
1,1970,3600200,1331.261758
2,1971,4218650,1265.488889
3,1972,4880900,1603.616515
4,1973,3858700,1663.510772


In [17]:
df_national.to_csv("data/crop/national_yield_with_trend.csv", index=False)
df_province.to_csv("data/crop/province_yield_with_trend.csv", index=False)